In [ ]:
import numpy as np
# 수치 연산용 라이브러리
import pandas as pd
# 데이터프레임 형태로 데이터 관리
import matplotlib.pyplot as plt
# 이미지 시각화용

import torch
# 텐서 및 GPU 연산 지원
import torch.nn as nn
# 신경망 구성 요소 (Linear, ReLU 등 포함)
import torch.optim as optim
# 최적화 알고리즘 (Adam 등)
from tqdm.notebook import tqdm_notebook
# Colab에서 학습 진행률 표시

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# 구글 드라이브 파일에 접근 가능하게 함

In [ ]:
def show(img): ##visualize your image
    plt.imshow(img[1:].reshape(28, 28), cmap='gray')
    plt.show()
    # 첫 번째 값(레이블)을 제외한 784개의 픽셀값을 28x28로 reshape하여 흑백 이미지로 보여줌

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/mnist_train.csv')
pxl = ["pxl{}".format(i) for i in range(len(df.columns)-1)]
df.columns = ["value"] + pxl
#df.rename(columns = {"5":"value"}, inplace = True)
# MNIST 훈련 데이터셋을 로드하고, 첫 번째 열은 value(정답 레이블), 나머지는 pxl0 ~ pxl783으로 이름 설정

In [ ]:
df.head(10)

In [ ]:
show(df.iloc[3].values) #visualize 3rd dataset (hand writing image)
# 첫 10개의 데이터를 확인하고, 3번째 손글씨 이미지를 시각화

In [ ]:
A = pd.DataFrame(df, columns = pxl).values
# 입력 데이터 (784 픽셀)
y_data = pd.DataFrame(df, columns = ["value"]).values
# 정답 숫자 (0~9)

In [ ]:
A.shape

In [ ]:
def digit_to_vec(value):
  vec = np.zeros(shape = 10)
  vec[value] = 1
  return vec

In [ ]:
y_data_vec = [digit_to_vec(y_data[k]) for k in range(len(y_data))]
B = np.array(y_data_vec)
# 전체 정답을 one-hot 인코딩으로 변환하여 B에 저장

In [ ]:
B

In [ ]:
class FCNN(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(FCNN, self).__init__()
        self.linear1 = nn.Linear(784, 64)
        self.linear2 = nn.Linear(64, 32)
        self.linear3 = nn.Linear(32, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_rate)
        self.softmax = nn.Softmax(dim=1)
        # 3개의 Linear층과 ReLU, Dropout, Softmax 사용
        # 입력 → 784, 출력 → 10 (클래스 수)

    def forward(self, x):
        #x = self.dropout(x)  # Dropout for input layer
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)  # Dropout after first ReLU

        x = self.linear2(x)
        x = self.relu(x)
        x = self.dropout(x)  # Dropout after second ReLU

        x = self.linear3(x)
        x = self.softmax(x)
        return x
    # 순전파 정의 (Dropout은 학습 시 과적합 방지 목적)

In [ ]:
model = FCNN()
# 모델 생성

In [ ]:
x = torch.tensor(A, dtype=torch.float32)
y = torch.tensor(B, dtype=torch.float32)
# NumPy 배열 → PyTorch 텐서 변환

In [ ]:
model(x)

In [ ]:
x.shape

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# 옵티마이저: Adam
loss_fn = nn.MSELoss()
# 손실 함수: 평균 제곱 오차 (MSE) → 분류 문제에서는 일반적으로 CrossEntropyLoss를 더 자주 사용
num_epochs = 1000


In [ ]:
for epoch in tqdm_notebook(range(num_epochs)):
    model.train()

    # Forward
    output = model(x)
    loss = loss_fn(output, y)

    # Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
      print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item():.4f}")
      # 학습단계: 순전파 → 손실 계산 → 역전파 → 파라미터 업데이트
      # 10 에폭마다 loss 출력

In [ ]:
df2=pd.read_csv('/content/drive/MyDrive/mnist_test.csv')
pxl = ["pxl{}".format(i) for i in range(len(df.columns)-1)]
df2.columns = ["value"] + pxl
A2 = pd.DataFrame(df2, columns = pxl).values
x_test = torch.tensor(A2, dtype=torch.float32)
# 테스트 데이터셋을 로드하고 입력만 추출해서 x_test 텐서로 변환

y_data2 = pd.DataFrame(df2, columns = ["value"]).values
y_data2 = y_data2.reshape(-1)
# 테스트 정답 라벨 추출 및 1차원 배열로 reshape

In [ ]:
model.eval()
y_pred = model(x_test).detach().cpu().numpy()
y_pred = np.argmax(y_pred,axis=1)
# 모델을 평가 모드로 전환 후 예측 수행
# 예측 결과는 one-hot 확률 벡터 → 가장 높은 확률의 인덱스로 클래스 결정

In [ ]:
y_pred

In [ ]:
y_data2

In [ ]:
(y_pred == y_data2).sum() / len(y_data2)
# 전체 중 예측이 정답과 일치하는 비율 (정확도) 계산